In [31]:
import pandas as pd
import numpy as np
import netCDF4 as nc

In [32]:
# 处理 CSV 文件的函数
def process_csv(file_path,Lat1,Lon1,Time1):
    df = pd.read_csv(file_path,encoding='gb2312')
    df = df[['时间','经度', '纬度','水温2']]
    # 剔除异常点
    index_temp = (df['水温2'] < 40.0)&(df['水温2'] > -2.5)
    df = df[index_temp]
    # 判断是否是第 1个csv
    if Lat1==0:
        current_u = np.nan; current_v = np.nan; 
    else:
        lon = np.array(df['经度'].iloc[-1]); lat = np.array(df['纬度'].iloc[-1])
        # 计算正北和正东方向距离
        from vincenty import vincenty
        from datetime import datetime
        before_pos = (Lat1,Lon1); current_pos = (np.array(lat),np.array(lat))
        dis_N = vincenty((Lat1,lon), (lat,lon))
        dis_E = vincenty((lat,Lon1), (lat,lon))
        # 计算两个时间的差值
        before_time = datetime.strptime(str(Time1), '%Y%m%d%H%M%S')
        current_time = datetime.strptime(df['时间'].iloc[-1], '%Y/%m/%d %H:%M:%S')
        time_difference = current_time - before_time
        time_interval = time_difference.total_seconds()# 总秒数差,float类型
        current_u = dis_E*1000/time_interval # 正东方向流速
        current_v = dis_N*1000/time_interval # 正北方向流速
    df['current_u'] = current_u
    df['current_v'] = current_v
    df['时间'] = pd.to_datetime(df['时间'])
    # 将日期时间转换为所需的数字格式  2024-04-20 05:31:00 -> 20240420053100
    df['时间'] = df['时间'].dt.strftime('%Y%m%d%H%M%S').astype('int64')
    Lat1 = df['纬度'].iloc[-1]; Lon1 = df['经度'].iloc[-1]; Time1 = df['时间'].iloc[-1]
    return Lon1,Lat1,Time1,df


In [33]:
# Lat1 = 0; Lon1 = 0; Time1 = 20240419053101
# # Lon1 = 111.390016; Lat1 = 17.030150; Time1 = 20240419032901
# file_path = r'C:\Users\lenovo\Desktop\Floater\Floater_QS_002_202303270530.csv'
# Lon1,Lat1,Time1,df = process_csv(file_path,Lat1,Lon1,Time1)

In [34]:
# print(Lon1)
# print(Lat1)
# print(Time1)

In [35]:
def nc_generate(df,new_path,file_path,lon2 = 105.21,lat2 = 15.11,ReleaseTime = '2022-12-11 14:03:07 UTC',ReleaseLongitude = '111.8444°E',ReleaseLatitude = '17.4169°N',Sensor_SN = 2408,Comment = 'quality control data',information ='') :
    # Floater_Type:'BS' lon2:105.21 lat2:15.11 deploy_time:202303270530(整数就可以，字符串也可以)
    name = file_path.split('\\')[-1] # 'AWG_HYY_003_202303270530.csv'
    Floater_Type = name.split('_')[1]
    deviceID = name.split('_')[2]
    
    time = df['时间'].iloc[-1]
    longitude = np.round(df['经度'].iloc[-1],4); latitude =np.round(df['纬度'].iloc[-1],4); 
    sst = np.round(df['水温2'].iloc[-1],4)
    current_u = np.round(df['current_u'].iloc[-1],2)
    current_v = np.round(df['current_v'].iloc[-1],2)
    
    num_row = df.shape[0] # 获取数据行数，为 node_dim准备
    # 创建nc文件
    format = 'NETCDF3_CLASSIC'
    last_time = str(time)[:12]
    filename = f'Floater_{Floater_Type}_{deviceID}_{lon2}E_{lat2}N_{last_time}_Q.nc'
    file  = nc.Dataset(new_path+'/'+f'{filename}', 'w',format= format)

    # 创建维度
    time_dim = file.createDimension('num_time',1) 
    lon_dim = file.createDimension('num_longitude',1)
    lat_dim = file.createDimension('num_latitude',1)
    node_dim = file.createDimension('num_node',1)

    # 创建变量
    time_var = file.createVariable('time','f8',('num_time',))
    lon_var = file.createVariable('longitude','f4',('num_longitude',))
    lat_var = file.createVariable('latitude','f4',('num_latitude',))
        # 'Latitude': 这是新创建的变量的名称
        # 'f4': 这指定了变量的数据类型。'f8'代表64位浮点数（双精度）
        # ('Latitude',):指定新变量所依赖的维度，('Latitude',)创建元组，而 ('Latitude')为一个字符串
    sst_var = file.createVariable('sst','f4',('num_node',))
    current_u_var = file.createVariable('current_u','f4',('num_node',))
    current_v_var = file.createVariable('current_v','f4',('num_node',))

    # 添加属性
    time_var.long_name="the UTC time observed by the floater"
    lon_var.long_name = 'the longitude located by the floater' 
    lon_var.units = "degrees_east" 
    lat_var.long_name = 'the latitude located by the floater'
    lat_var.units = "degrees_north"
    sst_var.long_name = "surface temperature of seawater observed by floater"
    sst_var.units = "Celsius" 
    current_u_var.long_name = "eastward surface velocity of seawater inferred by floater"
    current_u_var.units = "m/s" 
    current_v_var.long_name = "northward surface velocity of seawater inferred by floater"
    current_v_var.units = "m/s" 
    
    # 变量中写入数据
        # 必须time_var[:] = time_data，不可以time_var= time_data
        # 使用createVariable方法创建一个变量后，这个变量实际上是一个特殊的对象，它代表netCDF文件中的一个数据区域。
        # time_var = time_data会改变time_var的引用，而不是修改它所代表的数据区域的内容
        # 相反，time_var[:]是用来引用该变量整个数据区域的一个“切片”
        # time_var[:] = time_data 实际上是将time_data数组中的数据复制到time_var所代表的数据区域中
    time_var[:] = time
    lon_var[:] = longitude
    lat_var[:] = latitude
    sst_var[:] = sst
    current_u_var[:] = current_u
    current_v_var[:] = current_v

    # 全局属性(需要放到最后来，不然添加不上全局属性)
    file.Floater_SN = Floater_Type + '_' + deviceID
    file.Floater_ReleaseTime = ReleaseTime
    file.Floater_ReleaseLongitude = ReleaseLongitude
    file.Floater_ReleaseLatitude = ReleaseLatitude
    file.Sensor_SN = Sensor_SN
    file.Sensor_accuracy = 'temperature,OST71,±0.01℃'
    file.Comment = Comment
    file.Information = information
    file.Manufacturer = '国家海洋技术中心'
    file.Source = 'Wei Ma, TianJin University, wei.ma@tju.edu.cn'
    import datetime
    import pytz
    current_time = datetime.datetime.now() # 获取当前时间
    utc_time = current_time.astimezone(pytz.utc) # 将当前时间转换为 UTC 时间
    formatted_utc_time = utc_time.strftime("%Y-%m-%d %H:%M:%S %Z") # 格式化并显示 UTC 时间
    time_now = str(formatted_utc_time)[:19]
    file.History = f'Created by Python at {time_now} UTC'

    file.close() # 关闭文件
    nc_path = new_path+'/'+f'{filename}'
    return nc_path.replace('\\', '/')

In [36]:
Lat1 = 0; Lon1 = 0; Time1 = 0

In [37]:
print(Lon1)
print(Lat1)
print(Time1)

0
0
0


In [38]:
file_path = r'C:\Users\lenovo\Desktop\Floater\Floater_QS_003_202303270630.csv'

# Lon1 = 111.390016; Lat1 = 17.030150; Time1 = 20240419032901
Lon1,Lat1,Time1,df = process_csv(file_path,Lat1,Lon1,Time1)
# new_path = r'C:\Users\lenovo\Desktop\java\Glider05'
new_path = r'C:\Users\lenovo\Desktop\Floater' #'C:/Users/lenovo/Desktop/java/Glider05'  'C:/Users/lenovo/Desktop/任务帧nc数据'
Glider_Type = 'HY-BX' ;lon2 = 105.21 ;lat2 = 15.11 
nc_generate(df,new_path,file_path,lon2,lat2)

'C:/Users/lenovo/Desktop/Floater/Floater_QS_003_105.21E_15.11N_202404220631_Q.nc'

In [39]:
df

,时间,经度,纬度,水温2,current_u,current_v
0,20240422063103,112.690016,18.33015,28,NaN,NaN


In [40]:
print(Lon1)
print(Lat1)
print(Time1)

112.690016
18.33015
20240422063103


In [41]:
df

,时间,经度,纬度,水温2,current_u,current_v
0,20240422063103,112.690016,18.33015,28,NaN,NaN
